# Assignment 2

Support Code

In [1]:
# You are expected to use this support code.
# You may want to write:
# from ct_support_code import *
# at the top of your answers

# You will need NumPy and SciPy:
from ct_support_code import *
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import cho_factor, cho_solve


def params_unwrap(param_vec, shapes, sizes):
    """Helper routine for minimize_list"""
    args = []
    pos = 0
    for i in range(len(shapes)):
        sz = sizes[i]
        args.append(param_vec[pos:pos+sz].reshape(shapes[i]))
        pos += sz
    return args


def params_wrap(param_list):
    """Helper routine for minimize_list"""
    param_list = [np.array(x) for x in param_list]
    shapes = [x.shape for x in param_list]
    sizes = [x.size for x in param_list]
    param_vec = np.zeros(sum(sizes))
    pos = 0
    for param in param_list:
        sz = param.size
        param_vec[pos:pos+sz] = param.ravel()
        pos += sz
    unwrap = lambda pvec: params_unwrap(pvec, shapes, sizes)
    return param_vec, unwrap


def minimize_list(cost, init_list, args):
    """Optimize a list of arrays (wrapper of scipy.optimize.minimize)

    The input function "cost" should take a list of parameters,
    followed by any extra arguments:
        cost(init_list, *args)
    should return the cost of the initial condition, and a list in the same
    format as init_list giving gradients of the cost wrt the parameters.

    The options to the optimizer have been hard-coded. You may wish
    to change disp to True to get more diagnostics. You may want to
    decrease maxiter while debugging. Although please report all results
    in Q2-5 using maxiter=500.
    """
    opt = {'maxiter': 500, 'disp': False}
    init, unwrap = params_wrap(init_list)
    def wrap_cost(vec, *args):
        E, params_bar = cost(unwrap(vec), *args)
        vec_bar, _ = params_wrap(params_bar)
        return E, vec_bar
    res = minimize(wrap_cost, init, args, 'L-BFGS-B', jac=True, options=opt)
    return unwrap(res.x)


def linreg_cost(params, X, yy, alpha):
    """Regularized least squares cost function and gradients

    Can be optimized with minimize_list -- see fit_linreg_gradopt for a
    demonstration.

    Inputs:
    params: tuple (ww, bb): weights ww (D,), bias bb scalar
         X: N,D design matrix of input features
        yy: N,  real-valued targets
     alpha: regularization constant

    Outputs: (E, [ww_bar, bb_bar]), cost and gradients
    """
    # Unpack parameters from list
    ww, bb = params

    # forward computation of error
    ff = np.dot(X, ww) + bb
    res = ff - yy
    E = np.dot(res, res) + alpha*np.dot(ww, ww)

    # reverse computation of gradients
    ff_bar = 2*res
    bb_bar = np.sum(ff_bar)
    ww_bar = np.dot(X.T, ff_bar) + 2*alpha*ww

    return E, [ww_bar, bb_bar]


def fit_linreg_gradopt(X, yy, alpha):
    """
    fit a regularized linear regression model with gradient opt

         ww, bb = fit_linreg_gradopt(X, yy, alpha)

     Find weights and bias by using a gradient-based optimizer
     (minimize_list) to improve the regularized least squares cost:

       np.sum(((np.dot(X,ww) + bb) - yy)**2) + alpha*np.dot(ww,ww)

     Inputs:
             X N,D design matrix of input features
            yy N,  real-valued targets
         alpha     scalar regularization constant

     Outputs:
            ww D,  fitted weights
            bb     scalar fitted bias
    """
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array(0))
    ww, bb = minimize_list(linreg_cost, init, args)
    return ww, bb


def logreg_cost(params, X, yy, alpha):
    """Regularized logistic regression cost function and gradients

    Can be optimized with minimize_list -- see fit_linreg_gradopt for a
    demonstration of fitting a similar function.

    Inputs:
    params: tuple (ww, bb): weights ww (D,), bias bb scalar
         X: N,D design matrix of input features
        yy: N,  real-valued targets
     alpha: regularization constant

    Outputs: (E, [ww_bar, bb_bar]), cost and gradients
    """
    # Unpack parameters from list
    ww, bb = params

    # Force targets to be +/- 1
    yy = 2*(yy==1) - 1

    # forward computation of error
    aa = yy*(np.dot(X, ww) + bb)
    sigma = 1/(1 + np.exp(-aa))
    E = -np.sum(np.log(sigma)) + alpha*np.dot(ww, ww)

    # reverse computation of gradients
    aa_bar = sigma - 1
    bb_bar = np.dot(aa_bar, yy)
    ww_bar = np.dot(X.T, yy*aa_bar) + 2*alpha*ww

    return E, (ww_bar, bb_bar)


def nn_cost(params, X, yy=None, alpha=None):
    """NN_COST simple neural network cost function and gradients, or predictions

           E, params_bar = nn_cost([ww, bb, V, bk], X, yy, alpha)
                    pred = nn_cost([ww, bb, V, bk], X)

     Cost function E can be minimized with minimize_list

     Inputs:
             params (ww, bb, V, bk), where:
                    --------------------------------
                        ww K,  hidden-output weights
                        bb     scalar output bias
                         V K,D hidden-input weights
                        bk K,  hidden biases
                    --------------------------------
                  X N,D input design matrix
                 yy N,  regression targets
              alpha     scalar regularization for weights

     Outputs:
                     E  sum of squares error
            params_bar  gradients wrt params, same format as params
     OR
               pred N,  predictions if only params and X are given as inputs
    """
    # Unpack parameters from list
    ww, bb, V, bk = params

    # Forwards computation of cost
    A = np.dot(X, V.T) + bk[None,:] # N,K
    P = 1 / (1 + np.exp(-A)) # N,K
    F = np.dot(P, ww) + bb # N,
    if yy is None:
        # user wants prediction rather than training signal:
        return F
    res = F - yy # N,
    E = np.dot(res, res) + alpha*(np.sum(V*V) + np.dot(ww,ww)) # 1x1

    # Reverse computation of gradients
    F_bar = 2*res # N,
    ww_bar = np.dot(P.T, F_bar) + 2*alpha*ww # K,
    bb_bar = np.sum(F_bar) # scalar
    P_bar = np.dot(F_bar[:,None], ww[None,:]) # N,K
    A_bar = P_bar * P * (1 - P) # N,K
    V_bar = np.dot(A_bar.T, X) + 2*alpha*V # K,D
    bk_bar = np.sum(A_bar, 0)

    return E, (ww_bar, bb_bar, V_bar, bk_bar)


def rbf_fn(X1, X2):
    """Helper routine for gp_post_par"""
    return np.exp((np.dot(X1,(2*X2.T))-np.sum(X1*X1,1)[:,None]) - np.sum(X2*X2,1)[None,:])


def gauss_kernel_fn(X1, X2, ell, sigma_f):
    """Helper routine for gp_post_par"""
    return sigma_f**2 * rbf_fn(X1/(np.sqrt(2)*ell), X2/(np.sqrt(2)*ell))


def gp_post_par(X_rest, X_obs, yy, sigma_y=0.05, ell=5.0, sigma_f=0.1):
    """GP_POST_PAR means and covariances of a posterior Gaussian process

         rest_cond_mu, rest_cond_cov = gp_post_par(X_rest, X_obs, yy)
         rest_cond_mu, rest_cond_cov = gp_post_par(X_rest, X_obs, yy, sigma_y, ell, sigma_f)

     Calculate the means and covariances at all test locations of the posterior Gaussian
     process conditioned on the observations yy at observed locations X_obs.

     Inputs:
                 X_rest GP test locations
                  X_obs locations of observations
                     yy observed values
                sigma_y observation noise standard deviation
                    ell kernel function length scale
                sigma_f kernel function standard deviation

     Outputs:
           rest_cond_mu mean at each location in X_rest
          rest_cond_cov covariance matrix between function values at all test locations
    """
    X_rest = X_rest[:, None]
    X_obs = X_obs[:, None]
    K_rest = gauss_kernel_fn(X_rest, X_rest, ell, sigma_f)
    K_rest_obs = gauss_kernel_fn(X_rest, X_obs, ell, sigma_f)
    K_obs = gauss_kernel_fn(X_obs, X_obs, ell, sigma_f)
    M = K_obs + sigma_y**2 * np.eye(yy.size)
    M_cho, M_low = cho_factor(M)
    rest_cond_mu = np.dot(K_rest_obs, cho_solve((M_cho, M_low), yy))
    rest_cond_cov = K_rest - np.dot(K_rest_obs, cho_solve((M_cho, M_low), K_rest_obs.T))

    return rest_cond_mu, rest_cond_cov


Preliminary Data Exploration

In [2]:
from ct_support_code import *
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import cho_factor, cho_solve

# Load the data. 
data = np.load('data/ct_data.npz')

# Train, validation, and test sets... Do not shuffle the data further. 
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

In [3]:
# The input feature vectors are 384 dimensional. The target variable is cts. 
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}, X_val: {X_val.shape}, y_val: {y_val.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

X_train: (40754, 384), y_train: (40754,), X_val: (5785, 384), y_val: (5785,), X_test: (6961, 384), y_test: (6961,)


In [4]:
X_train[0] # The input feature vectors are normalized between 0 and 1. Additionally, they're quite sparse.

array([ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
       -0.25    , -0.25    , -0.25    , -0.25    ,  0.      ,  0.      ,
        0.      ,  0.8     ,  0.      ,  0.      ,  0.      , -0.25    ,
       -0.25    , -0.25    ,  0.      ,  0.      ,  0.      ,  0.956533,
        0.8521  ,  0.      ,  0.      ,  0.      ,  0.      , -0.25    ,
        0.      ,  0.      ,  0.      ,  0.932476,  0.960794,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.791258,  0.979383,  0.8725  ,  0.      ,  0.      ,
        0.      , -0.25    ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.16    ,  0.979998,  0.954111,  0.      , -0.25    , -0.25    ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.689167,  0.987036,
        0.92411 ,  0.      , -0.25    , -0.25    ,  0.      ,  0.      ,
        0.      ,  0.867052,  0.975287,  0.813456,  0.      ,  0.      ,
        0.      , -0.25    ,  0.      ,  0.      , 

In [5]:
# Max, min, average value of y_train, st dev of y_train
print(f"Max: {np.max(y_train)}, Min: {np.min(y_train)}, Mean: {np.mean(y_train)}, Std Dev: {np.std(y_train)}")
# The target variable follows a standard normal distribution with mean 0 and standard deviation 1.

Max: 2.2265180851800834, Min: -1.8679386519531087, Mean: -9.13868774539957e-15, Std Dev: 0.9999877311903766


In [6]:
# What is the train/validation/test split? in terms of percentages. 
print(f"Train: {X_train.shape[0]/(X_train.shape[0] + X_val.shape[0] + X_test.shape[0])}, Validation: {X_val.shape[0]/(X_train.shape[0] + X_val.shape[0] + X_test.shape[0])}, Test: {X_test.shape[0]/(X_train.shape[0] + X_val.shape[0] + X_test.shape[0])}")

Train: 0.7617570093457944, Validation: 0.10813084112149533, Test: 0.13011214953271028


The dataset consists of 384 features extracted from CT images. The class variable is numeric and denotes the relative location of the CT slice on the axial axis of the human body.

* What do the features represent?
    * 1 - 240.:         Histogram describing bone structures
    * 241 - 384.:       Histogram describing air inclusions

Each CT slice is described by two histograms in polar space.
The first histogram describes the location of bone structures in the image,
the second the location of air inclusions inside of the body.
Both histograms are concatenated to form the final feature vector.
Bins that are outside of the image are marked with the value -0.25.

The class variable (relative location of an image on the axial axis) was
constructed by manually annotating up to 10 different distinct landmarks in
each CT Volume with known location. The location of slices in between
landmarks was interpolated.

The data was retrieved from a set of 53500 CT images from 74 different
patients (43 male, 31 female). The first 73 patients were put in the _train arrays, the next 12 in the _val arrays, and the final 12 in the _test arrays. Please use this training, validation, test split as given. Do not shuffle the data further in this assignment.

### Note: Your answers should be executable and should PRINT requested information. Also, manually include as part of worded answer.

In [7]:
# Verify that (up to numerical rounding errors) the mean of the training positions in y_train is zero. 
sample_mean_y_train = np.mean(y_train)
print(sample_mean_y_train)

# The mean of the 5,785 positions in the y_val array is not zero.
sample_mean_y_val = np.mean(y_val)
print(sample_mean_y_val) 

-9.13868774539957e-15
-0.2160085093241599


Load the data into Python (your code can assume this step has been done):

* Verify that (up to numerical rounding errors) the mean of the training positions in y_train is zero. The mean of the 5,785 positions in the y_val array is not zero. Report its mean with a “standard error”, temporarily assuming that each entry is independent. For comparison, also report the mean with a standard error of the first 5,785 entries in the y_train. Explain how your results demonstrate that these standard error bars do not reliably indicate what the average of locations in future CT slice data will be. Why are standard error bars misleading here?

# Question 1: Getting Started

In [8]:
import numpy as np
# Verify that (up to numerical rounding errors) the mean of the training positions in y_train is zero.
sample_mean_y_train = np.mean(y_train)
print(f"Sample mean of y_train: {sample_mean_y_train}")

# The mean of the 5,785 positions in the y_val array is not zero.
sample_mean_y_val = np.mean(y_val)
print(f"Sample mean of y_val: {sample_mean_y_val}")

# Calculate the sample mean and standard error of the first 5785 positions in y_train
truncated_sample_mean_y_train = np.mean(y_train[:5785])
truncated_sample_std_y_train = np.std(y_train[:5785], ddof=1)  # using ddof=1 for sample std deviation
N = 5785
truncated_st_error_y_train = truncated_sample_std_y_train / np.sqrt(N)
print(f"Sample mean of first 5785 positions in y_train: {truncated_sample_mean_y_train}")
print(f"Sample std dev of first 5785 positions in y_train: {truncated_sample_std_y_train}")
print(f"Standard error of first 5785 positions in y_train: {truncated_st_error_y_train}")

# Calculate the sample standard deviation and standard error of y_val
sample_std_y_val = np.std(y_val, ddof=1)  # using ddof=1 for sample std deviation
std_error_y_val = sample_std_y_val / np.sqrt(N)
print(f"Sample mean of y_val: {sample_mean_y_val}")
print(f"Sample std dev of y_val: {sample_std_y_val}")
print(f"Standard error of y_val: {std_error_y_val}")


Sample mean of y_train: -9.13868774539957e-15
Sample mean of y_val: -0.2160085093241599
Sample mean of first 5785 positions in y_train: -0.44247687859693674
Sample std dev of first 5785 positions in y_train: 0.9071810045985477
Standard error of first 5785 positions in y_train: 0.011927303389170828
Sample mean of y_val: -0.2160085093241599
Sample std dev of y_val: 0.9815056935674723
Standard error of y_val: 0.01290449880016868


Some of the input features are constants: they take on the same value for every training example. Identify these features, and remove them from the input matrices in the training, validation, and testing sets.

In [9]:
import numpy as np

# Function to remove constant features
def remove_constant_features(X_train, X_val, X_test):
    
    # Identify constant columns
    constant_mask = np.all(X_train == X_train[0, :], axis=0)  # True for constant columns
    constant_indices = np.where(constant_mask)[0]  # Indices of constant columns
    
    # Remove these columns from all datasets
    X_train = X_train[:, ~constant_mask]
    X_val = X_val[:, ~constant_mask]
    X_test = X_test[:, ~constant_mask]
    
    return X_train, X_val, X_test, constant_indices

def remove_duplicate_features(X_train, X_val, X_test):
    # Initialize a list to store indices of unique columns
    unique_indices = []
    duplicate_indices = []
    seen_columns = {}
    
    for idx in range(X_train.shape[1]):
        # Convert column to a tuple (hashable type) for dictionary storage
        col_tuple = tuple(X_train[:, idx])
        if col_tuple not in seen_columns:
            seen_columns[col_tuple] = idx
            unique_indices.append(idx)
        else:
            duplicate_indices.append(idx)
    
    # Keep only unique columns in the original order
    X_train = X_train[:, unique_indices]
    X_val = X_val[:, unique_indices]
    X_test = X_test[:, unique_indices]
    
    return X_train, X_val, X_test, duplicate_indices


# Perform the cleaning steps
X_train, X_val, X_test, constant_indices = remove_constant_features(X_train, X_val, X_test)
X_train, X_val, X_test, duplicate_indices = remove_duplicate_features(X_train, X_val, X_test)

# Report removed columns
print("Removed constant feature indices:", constant_indices)
print("Removed duplicate feature indices:", duplicate_indices)

Removed constant feature indices: [ 59  69 179 189 351]
Removed duplicate feature indices: [76, 77, 185, 195, 283, 354]


In [10]:
print(f"New shapes: X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")

New shapes: X_train: (40754, 373), X_val: (5785, 373), X_test: (6961, 373)


# Question 2: Linear Regression

In [11]:
import numpy as np
from ct_support_code import *

def fit_linreg(X, yy, alpha):
    # Add a column of ones to X to account for the bias term
    X_augmented = np.hstack([X, np.ones((X.shape[0], 1))])
    
    # Augment X and yy for regularization
    reg_matrix = np.sqrt(alpha) * np.eye(X.shape[1] + 1)
    reg_matrix[-1, -1] = 0  # Do not regularize the bias term
    
    X_reg = np.vstack([X_augmented, reg_matrix])
    yy_reg = np.concatenate([yy, np.zeros(X.shape[1] + 1)])
    
    # Solve using lstsq to obtain the weights and bias
    params, _, _, _ = np.linalg.lstsq(X_reg, yy_reg, rcond=None)
    
    # Separate weights and bias
    w = params[:-1]
    b = params[-1]
    
    return w, b

# Root mean squared error
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

# Compute prediction vector
def predict_linreg(X, w, b):
    return np.dot(X, w) + b

# Fit the linear regression model
w, b = fit_linreg(X_train, y_train, 30)

# Fit with gradient-based optimizer 
ww, bb = fit_linreg_gradopt(X_train, y_train, 30)

# Compute predictions on validation set. 
y_pred = predict_linreg(X_val, w, b)
y_pred_opt = predict_linreg(X_val, ww, bb)

# Compute RMSE on validation set for both
rmse_val = rmse(y_val, y_pred)
rmse_val_opt = rmse(y_val, y_pred_opt)

# Compute RMSE on training set for both
rmse_train = rmse(y_train, predict_linreg(X_train, w, b))
rmse_train_opt = rmse(y_train, predict_linreg(X_train, ww, bb))

# Compare RMSE values
print(f"RMSE on training set (closed-form): {rmse_train}")
print(f"RMSE on training set (gradient-based): {rmse_train_opt}")
print(f"RMSE on validation set (closed-form): {rmse_val}")
print(f"RMSE on validation set (gradient-based): {rmse_val_opt}")

# See if the parameters are roughly the same
print(f"Weight difference: {np.linalg.norm(w - ww)}")
print(f"Bias difference: {np.abs(b - bb)}")

# Are they approximately the same? 1e-2 is a reasonable tolerance
print(f"Are the weights the same? {np.allclose(w, ww, atol=1e-2)}")
print(f"Are the biases the same? {np.allclose(b, bb, atol=1e-2)}")

RMSE on training set (closed-form): 0.3567565397204054
RMSE on training set (gradient-based): 0.35675704441316
RMSE on validation set (closed-form): 0.4230521968394692
RMSE on validation set (gradient-based): 0.4230514057395294
Weight difference: 0.003069446594797705
Bias difference: 0.0007478002387096144
Are the weights the same? True
Are the biases the same? True


# Question 3: Invented Classification Task

In [12]:
import numpy as np
from ct_support_code import *

# Root mean squared error
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

# Compute prediction vector
def predict_linreg(X, w, b):
    return np.dot(X, w) + b

K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)

# Initialize the transformed feature matrices
prob_train = np.zeros((X_train.shape[0], K))
prob_val = np.zeros((X_val.shape[0], K))

for kk in range(K):
    # Define binary labels based on threshold
    labels = (y_train > thresholds[kk])
    
    # Fit logistic regression model using gradient optimizer
    ww, bb = minimize_list(logreg_cost, (np.zeros(X_train.shape[1]), 0), (X_train, labels, 30))
    
    # Predict probabilities on training and validation sets
    prob_train[:, kk] = 1 / (1 + np.exp(-(np.dot(X_train, ww) + bb)))
    prob_val[:, kk] = 1 / (1 + np.exp(-(np.dot(X_val, ww) + bb)))

# Fit regularized linear regression on the 20-dimensional probability features
w, b = fit_linreg_gradopt(prob_train, y_train, 30)

# Compute predictions and RMSE on training and validation sets
train_preds = predict_linreg(prob_train, w, b)
val_preds = predict_linreg(prob_val, w, b)

rmse_train = rmse(y_train, train_preds)
rmse_val = rmse(y_val, val_preds)

print(f"RMSE on training set: {rmse_train}")
print(f"RMSE on validation set: {rmse_val}")


RMSE on training set: 0.15441142592866933
RMSE on validation set: 0.25424916988852914


# Question 4: Small Neural Network

In [16]:
import numpy as np
from ct_support_code import *

# # Root mean squared error
# def rmse(y_true, y_pred):
#     return np.sqrt(np.mean((y_true - y_pred) ** 2))

# # Compute prediction vector
# def predict_linreg(X, w, b):
#     return np.dot(X, w) + b

# K = 20 # number of thresholded classification problems to fit
# mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
# thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)

# # Initialize the transformed feature matrices
# prob_train = np.zeros((X_train.shape[0], K))
# prob_val = np.zeros((X_val.shape[0], K))

# for kk in range(K):
#     # Define binary labels based on threshold
#     labels = (y_train > thresholds[kk])
    
#     # Fit logistic regression model using gradient optimizer
#     ww, bb = minimize_list(logreg_cost, (np.zeros(X_train.shape[1]), 0), (X_train, labels, 30))
    
#     # Predict probabilities on training and validation sets
#     prob_train[:, kk] = 1 / (1 + np.exp(-(np.dot(X_train, ww) + bb)))
#     prob_val[:, kk] = 1 / (1 + np.exp(-(np.dot(X_val, ww) + bb)))

# # Fit regularized linear regression on the 20-dimensional probability features
# w, b = fit_linreg_gradopt(prob_train, y_train, 30)

# Random initialization of parameters
np.random.seed(42)  # For reproducibility
D = X_train.shape[1]  # Input dimensionality

random_params = [
    np.random.randn(K),  # ww: Output layer weights
    np.random.randn(),   # bb: Output layer bias
    np.random.randn(K, D),  # V: Hidden layer weights
    np.random.randn(K)   # bk: Hidden layer biases
]

# Initialization using Q3 fits
q3_params = [
# You must run the code in Q3 to get these parameters.  
    w,  # ww: Output layer weights from Q3
    b,  # bb: Output layer bias from Q3
    np.vstack([minimize_list(logreg_cost, (np.zeros(D), 0), (X_train, (y_train > t), 30))[0] for t in thresholds]),  # V: Hidden layer weights from Q3
    np.array([minimize_list(logreg_cost, (np.zeros(D), 0), (X_train, (y_train > t), 30))[1] for t in thresholds])  # bk: Hidden layer biases from Q3
]

# Fit the neural network with both initializations
alpha = 30  # Regularization constant

# Fit with random initialization
nn_random_params = minimize_list(nn_cost, random_params, (X_train, y_train, alpha))

# Fit with Q3 initialization
nn_q3_params = minimize_list(nn_cost, q3_params, (X_train, y_train, alpha))

# Predict on training and validation sets
nn_random_preds_train = nn_cost(nn_random_params, X_train)
nn_random_preds_val = nn_cost(nn_random_params, X_val)

nn_q3_preds_train = nn_cost(nn_q3_params, X_train)
nn_q3_preds_val = nn_cost(nn_q3_params, X_val)

# Compute RMSE for both initializations
rmse_random_train = rmse(y_train, nn_random_preds_train)
rmse_random_val = rmse(y_val, nn_random_preds_val)

rmse_q3_train = rmse(y_train, nn_q3_preds_train)
rmse_q3_val = rmse(y_val, nn_q3_preds_val)

# Print RMSE values
print(f"RMSE on training set (random init): {rmse_random_train}")
print(f"RMSE on validation set (random init): {rmse_random_val}")

print(f"RMSE on training set (Q3 init): {rmse_q3_train}")
print(f"RMSE on validation set (Q3 init): {rmse_q3_val}")


RMSE on training set (random init): 0.13839872700253528
RMSE on validation set (random init): 0.2688816261838681
RMSE on training set (Q3 init): 0.13950899960871216
RMSE on validation set (Q3 init): 0.26875475907458385


# Question 5, Attempt 2

In [17]:
import numpy as np
from scipy.stats import norm
from ct_support_code import *

K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
D = X_train.shape[1]  # Input dimensionality

# Initialization using Q3 fits
q3_params = [
    w,  # ww: Output layer weights from Q3
    b,  # bb: Output layer bias from Q3
    np.vstack([minimize_list(logreg_cost, (np.zeros(D), 0), (X_train, (y_train > t), 30))[0] for t in thresholds]),  # V: Hidden layer weights from Q3
    np.array([minimize_list(logreg_cost, (np.zeros(D), 0), (X_train, (y_train > t), 30))[1] for t in thresholds])  # bk: Hidden layer biases from Q3
]

# Root mean squared error
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

# Neural network training function
def train_nn_reg(alpha, X_eval, y_eval):
    nn_params = minimize_list(nn_cost, q3_params, (X_train, y_train, alpha))
    eval_preds = nn_cost(nn_params, X_eval)  # Ensure nn_cost returns predictions
    return rmse(y_eval, eval_preds)


In [ ]:
import numpy as np
from ct_support_code import *

# Root mean squared error function
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

# Number of networks in the ensemble
num_networks = 5
best_alpha = 3.64  # Best alpha from Bayesian optimization
K = 20             # Number of hidden units
D = X_train.shape[1]  # Input dimensionality

# Initialize parameters using Q3 fits
q3_params = [
    w,  # Output layer weights from Q3
    b,  # Output layer bias from Q3
    np.vstack([
        minimize_list(logreg_cost, (np.zeros(D), 0), (X_train, (y_train > t), 30))[0]
        for t in thresholds
    ]),  # Hidden layer weights from Q3
    np.array([
        minimize_list(logreg_cost, (np.zeros(D), 0), (X_train, (y_train > t), 30))[1]
        for t in thresholds
    ])   # Hidden layer biases from Q3
]

# Lists to store predictions
ensemble_preds_train = []
ensemble_preds_val = []
ensemble_preds_test = []

# Training multiple neural networks
for i in range(num_networks):
    # Different random seed for each network to introduce diversity
    np.random.seed(i)
    print(f"Training network {i+1}...")
    # Slight random perturbation to Q3 parameters for initialization
    init_params = [
        q3_params[0] + np.random.randn(K) * 0.05,      # ww: Output layer weights
        q3_params[1] + np.random.randn() * 0.05,       # bb: Output layer bias
        q3_params[2] + np.random.randn(K, D) * 0.05,   # V: Hidden layer weights
        q3_params[3] + np.random.randn(K) * 0.05       # bk: Hidden layer biases
    ]
    
    # Train neural network
    nn_params = minimize_list(nn_cost, init_params, (X_train, y_train, best_alpha))
    
    # Predict on training, validation, and test sets
    preds_train = nn_cost(nn_params, X_train)
    preds_val = nn_cost(nn_params, X_val)
    preds_test = nn_cost(nn_params, X_test)
    
    # Store predictions
    ensemble_preds_train.append(preds_train)
    ensemble_preds_val.append(preds_val)
    ensemble_preds_test.append(preds_test)

# Averaging predictions across the ensemble
ensemble_preds_train = np.mean(ensemble_preds_train, axis=0)
ensemble_preds_val = np.mean(ensemble_preds_val, axis=0)
ensemble_preds_test = np.mean(ensemble_preds_test, axis=0)

# Compute RMSE
rmse_train = rmse(y_train, ensemble_preds_train)
rmse_val = rmse(y_val, ensemble_preds_val)
rmse_test = rmse(y_test, ensemble_preds_test)

print(f"Baseline Model Validation RMSE: {0.2428:.4f}") # Precomputed in Q5.
print(f"Baseline Model Test RMSE: {0.2782:.4f}\n") # Precomputed in Q5.

print(f"Ensemble Validation RMSE: {rmse_val:.4f}")
print(f"Ensemble Test RMSE: {rmse_test:.4f}")


Training network 1...
Training network 2...
Training network 3...
Training network 4...
Training network 5...
Baseline Model Validation RMSE: 0.2428
Baseline Model Test RMSE: 0.2782

Ensemble Validation RMSE: 0.2370
Ensemble Test RMSE: 0.2658


In [ ]:
import numpy as np
from ct_support_code import *

# Root mean squared error function
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

# Number of networks in the ensemble
num_networks = 5
best_alpha = 6.64  # Best alpha from Bayesian optimization
K = 20             # Number of hidden units
D = X_train.shape[1]  # Input dimensionality

# Initialize parameters using Q3 fits
q3_params = [
    w,  # Output layer weights from Q3
    b,  # Output layer bias from Q3
    np.vstack([
        minimize_list(logreg_cost, (np.zeros(D), 0), (X_train, (y_train > t), 30))[0]
        for t in thresholds
    ]),  # Hidden layer weights from Q3
    np.array([
        minimize_list(logreg_cost, (np.zeros(D), 0), (X_train, (y_train > t), 30))[1]
        for t in thresholds
    ])   # Hidden layer biases from Q3
]

# Lists to store predictions
ensemble_preds_val = []
ensemble_preds_test = []

# Training multiple neural networks with bootstrapped datasets
for i in range(num_networks):
    # Bootstrapping: Resample the training data with replacement
    np.random.seed(i)
    indices = np.random.choice(len(X_train), len(X_train), replace=True)
    X_bootstrap = X_train[indices]
    y_bootstrap = y_train[indices]

    print(f"Training network {i+1} with bootstrapped data...")
    # Slight random perturbation to Q3 parameters for initialization
    init_params = [
        q3_params[0] + np.random.randn(K) * 0.05,      # ww: Output layer weights
        q3_params[1] + np.random.randn() * 0.05,       # bb: Output layer bias
        q3_params[2] + np.random.randn(K, D) * 0.05,   # V: Hidden layer weights
        q3_params[3] + np.random.randn(K) * 0.05       # bk: Hidden layer biases
    ]
    
    # Train neural network on the bootstrapped dataset
    nn_params = minimize_list(nn_cost, init_params, (X_bootstrap, y_bootstrap, best_alpha))
    
    # Predict on validation and test sets
    preds_val = nn_cost(nn_params, X_val)
    preds_test = nn_cost(nn_params, X_test)
    
    # Store predictions
    ensemble_preds_val.append(preds_val)
    ensemble_preds_test.append(preds_test)

# Averaging predictions across the ensemble
ensemble_preds_val = np.mean(ensemble_preds_val, axis=0)
ensemble_preds_test = np.mean(ensemble_preds_test, axis=0)

# Compute RMSE
rmse_val = rmse(y_val, ensemble_preds_val)
rmse_test = rmse(y_test, ensemble_preds_test)

print(f"Baseline Model Validation RMSE: {0.2428:.4f}") # Precomputed in Q5.
print(f"Baseline Model Test RMSE: {0.2782:.4f}\n") # Precomputed in Q5.

print(f"Ensemble Validation RMSE (bootstrapping): {rmse_val:.4f}")
print(f"Ensemble Test RMSE (bootstrapping): {rmse_test:.4f}")


Training network 1...
Training network 2...
Training network 3...
Training network 4...
Training network 5...
Baseline Model Validation RMSE: 0.2428
Baseline Model Test RMSE: 0.2782

Ensemble Validation RMSE: 0.2632
Ensemble Test RMSE: 0.2949


In [25]:
import numpy as np
from ct_support_code import *

# Root mean squared error function
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

# Number of networks in the ensemble
num_networks = 5
best_alpha = 6.64  # Best alpha from Bayesian optimization
K = 20             # Number of hidden units
D = X_train.shape[1]  # Input dimensionality

# Initialize parameters using Q3 fits
q3_params = [
    # READ: You must run the code in Q3 to get these parameters w and b!
    w,  # Output layer weights from Q3
    b,  # Output layer bias from Q3
    np.vstack([
        minimize_list(logreg_cost, (np.zeros(D), 0), (X_train, (y_train > t), 30))[0]
        for t in thresholds
    ]),  # Hidden layer weights from Q3
    np.array([
        minimize_list(logreg_cost, (np.zeros(D), 0), (X_train, (y_train > t), 30))[1]
        for t in thresholds
    ])   # Hidden layer biases from Q3
]

# Lists to store predictions
ensemble_preds_val = []
ensemble_preds_test = []

# Training multiple neural networks with bootstrapped datasets
for i in range(num_networks):
    # Bootstrapping: Resample the training data with replacement
    np.random.seed(420+i)
    indices = np.random.choice(len(X_train), len(X_train), replace=True)
    X_bootstrap = X_train[indices]
    y_bootstrap = y_train[indices]

    print(f"Training network {i+1} with bootstrapped data...")
    # Slight random perturbation to Q3 parameters for initialization
    init_params = [
        q3_params[0] + np.random.randn(K) * 0.05,      # ww: Output layer weights
        q3_params[1] + np.random.randn() * 0.05,       # bb: Output layer bias
        q3_params[2] + np.random.randn(K, D) * 0.05,   # V: Hidden layer weights
        q3_params[3] + np.random.randn(K) * 0.05       # bk: Hidden layer biases
    ]
    
    # Train neural network on the bootstrapped dataset
    nn_params = minimize_list(nn_cost, init_params, (X_bootstrap, y_bootstrap, best_alpha))
    
    # Predict on validation and test sets
    preds_val = nn_cost(nn_params, X_val)
    preds_test = nn_cost(nn_params, X_test)
    
    # Store predictions
    ensemble_preds_val.append(preds_val)
    ensemble_preds_test.append(preds_test)

# Averaging predictions across the ensemble
ensemble_preds_val = np.mean(ensemble_preds_val, axis=0)
ensemble_preds_test = np.mean(ensemble_preds_test, axis=0)

# Compute RMSE
rmse_val = rmse(y_val, ensemble_preds_val)
rmse_test = rmse(y_test, ensemble_preds_test)

print(f"Baseline Model Validation RMSE: {0.2380:.4f}") # Precomputed in Q5.
print(f"Baseline Model Test RMSE: {0.2858:.4f}\n") # Precomputed in Q5.

print(f"Ensemble Validation RMSE (with bootstrapping): {rmse_val:.4f}")
print(f"Ensemble Test RMSE (with bootstrapping): {rmse_test:.4f}")


Training network 1 with bootstrapped data...
Training network 2 with bootstrapped data...
Training network 3 with bootstrapped data...
Training network 4 with bootstrapped data...
Training network 5 with bootstrapped data...
Baseline Model Validation RMSE: 0.2380
Baseline Model Test RMSE: 0.2858

Ensemble Validation RMSE (with bootstrapping): 0.2478
Ensemble Test RMSE (with bootstrapping): 0.2766
